# MMA Offshore’s PE

**This project investigates the P/E ratio of an ASX-listed company to gain insight into the approach used by asset managers.**

### Section 1
> *Create a simple profit and loss account (using the same line-items as the company does) in Excel for the last 3 full financial years. Focus on continuing operations (we will assume the company is honest about what is a ‘one-off’ item of expense or income). Use Excel formulas to calculate sub-totals and totals.*

Financials data has been collected from annual reports retrieved from the company website: https://www.mmaoffshore.com/investor-centre/financial-reports

Profit and loss accounts for continuing operations has stored in a CSV file: https://github.com/lei-per/MMA/blob/master/financials.csv

The script below reads the data from the CSV file into a dataframe in R.

In [1]:
# Read data from CSV file
financials = read.csv("financials.csv", row.names = 1, header = TRUE)
financials

,FY17,FY18,FY19
revenue,221766,200444,239259
invest_income,133,463,1278
other,-14960,87,-556
vessel_expenses,-241636,-206484,-238951
admin_expenses,-7377,-7092,-7402
impairment,-287542,8407,-10361
finance_costs,-26444,-23201,-19146
income_tax,1729,-533,-1494


Now that the data has been imported, we can calculate the total profit or loss for each financial year (`profit_loss_total`).

The weighted average number of ordinary shares used in the calculation of earnings per share (`ordinary_shares`) has been used to calculate the earnings per share (`earnings_per_share`). The `earnings_per_share` figures calculated match those in the company's annual reports.


The same calculations have also been repeated in Excel: https://github.com/lei-per/MMA/blob/master/financials.xlsx

The result show that the **earnings per share is negative in each year**. This makes sense as the company has reported an overall loss each year.


In [2]:
# Calculate profit or loss totals for each financial year
profit_loss_total <- colSums(financials[1:8, ])

# Set the number of ordinary shares used in the calculation of earnings per share
ordinary_shares <- c(402771, 678468, 858077)
names(ordinary_shares) <- c("FY17", "FY18", "FY19")

# Calculate earnings per share (in cents per share)
earnings_per_share <- round(profit_loss_total / ordinary_shares * 100, 3)

# Summarise results of the analysis in a dataframe
analysis <- data.frame(profit_loss_total, ordinary_shares, earnings_per_share)
analysis

,profit_loss_total,ordinary_shares,earnings_per_share
FY17,-354331,402771,-87.973
FY18,-27909,678468,-4.114
FY19,-37373,858077,-4.355


### Section 2
> *Add a single financial year forecast and assume last years revenue growth continues and all other income and expenses grow by 5%. Show your forecast profit for MRM on this basis.*

The script below forecasts the total profit or loss for the 2020 financial year (`profit_loss_total_FY20_forecast`).

Based on the assumptions above, a loss of **$4.87m** is forecast for MRM in the 2020 financial year.

In [3]:
# Calculate a forecast for the 2020 financial year by assuming all income and expenses grow by 5%
FY20_forecast <- round(1.05 * financials[1:8, "FY19"])
names(FY20_forecast) <- rownames(financials[1:8, ])

# Update the revenue forecast by using last years revenue growth rate
growth_rate <- financials["revenue", "FY19"] / financials["revenue", "FY18"]
FY20_forecast["revenue"] <- round(growth_rate * financials["revenue", "FY19"])
data.frame(FY20_forecast)

# Calculate profit or loss total for the 2020 financial year forecast
profit_loss_total_FY20_forecast <- sum(FY20_forecast)
names(profit_loss_total_FY20_forecast) <- "profit_loss_total_FY20_forecast"
profit_loss_total_FY20_forecast

,FY20_forecast
revenue,285590
invest_income,1342
other,-584
vessel_expenses,-250899
admin_expenses,-7772
impairment,-10879
finance_costs,-20103
income_tax,-1569


profit_loss_total_FY20_forecast 
                          -4874

### Section 3
> *Find historical share prices anywhere online for MRM and work out what approximate Price-to-Earnings ratio (PE) the company was trading at, 1 month after it reported for each of the last 2 years. Do you think this PE is a good way of valuing this company? (If not, can you suggest a different approach).*

The `quantmod` package in R has been used to retrieve stock data from yahoo finance.

In [5]:
# Load the quantmod package
library("quantmod")

In [7]:
# Use getSymbols from the quantmod package to get stock data from yahoo finance
MRM.AX <- getSymbols("MRM.AX", src = "yahoo", auto.assign = FALSE)

# Display the first and last three rows of the stock data
data.frame(head(MRM.AX, 3))

,MRM.AX.Open,MRM.AX.High,MRM.AX.Low,MRM.AX.Close,MRM.AX.Volume,MRM.AX.Adjusted
2007-01-02,0.86992,0.87436,0.84329,0.86992,79427,0.616371
2007-01-03,0.87436,0.87436,0.83442,0.86992,133652,0.616371
2007-01-04,0.86105,0.87436,0.84329,0.86992,93353,0.616371


Preliminary financial reports were released on 28th August 2017, 27th August 2018 and 26th August 2019. Share prices one month after each report have been used to calculate the price-earnings ratio (`pe_ratio`). 

In [8]:
# Subset stock data to select prices (in cents) one month after each report was released
price_per_share <- as.vector(MRM.AX[c("2017-09-28", "2018-09-27", "2019-09-26"), "MRM.AX.Adjusted"] * 100)

# Calculate PE ratios
pe_ratio <- round(price_per_share / earnings_per_share, 3)

# Add new columns to the analysis dataframe
analysis$price_per_share <- price_per_share
analysis$pe_ratio <- pe_ratio
analysis

,profit_loss_total,ordinary_shares,earnings_per_share,price_per_share,pe_ratio
FY17,-354331,402771,-87.973,17.585,-0.200
FY18,-27909,678468,-4.114,24.000,-5.834
FY19,-37373,858077,-4.355,19.000,-4.363


The result show the PE ratio is negative each year, at a multiple of -5.003 for FY17, then increasing substantially to -0.171 for FY18 and then decreasing slightly to -0.229 for FY19. In simplified terms, PE ratio can be thought of as the cost at which investors may obtain a given rate of corporate earnings. For this reason, PE ratio is commonly used to determine whether a company is overvalued (high PE) or undervalued (low PE) at its current share price. However, this relationship no longer holds when PE ratio is negative. For example, when the share price increased from 24 cents in FY18 to 19 cents in FY19, this decrease in the share price resulted in an ***increase*** to the PE ratio. For this reason, when the PE ratio is negative it is no longer a useful measure to compare value.

One alternative would be to calculate a forward PE ratio using forecast earnings. However, the forward PE ratio would not offer any advantage in this case, as the forecast for FY20 earnings is also negative (forecasted loss of $4.87m).

Another alternative is to use the Enterprise Value / EBITDA ratio as a measure of relative value.

In [9]:
# Add depreciation data to the profit and loss account data
depreciation <- c(45541, 31903, 35319)
financials["depreciation",] <- depreciation

# Calculate EBITDA 
ebitda <- colSums(financials[1:9, ]) - colSums(financials[6:8,])
data.frame(ebitda)

,ebitda
FY17,3467
FY18,19321
FY19,28947


In [10]:
# Add cash equivalents and non current liabilities to the profit and loss account data
cash_equivalents <- c(28757, 69648, 70155)
noncurrent_liabilities <- c(323929, 265215, 268255)
financials["cash_equivalents",] <- cash_equivalents
financials["noncurrent_liabilities",] <- noncurrent_liabilities
financials

# Calculate Enterprise Value 
enterprise_value <- round(ordinary_shares * price_per_share / 100 + noncurrent_liabilities - cash_equivalents)

# Add new columns to the analysis dataframe
analysis$ebitda <- ebitda
analysis$enterprise_value <- enterprise_value
analysis$ev_ebitda_ratio <- round(enterprise_value / ebitda, 3)
analysis

,FY17,FY18,FY19
revenue,221766,200444,239259
invest_income,133,463,1278
other,-14960,87,-556
vessel_expenses,-241636,-206484,-238951
admin_expenses,-7377,-7092,-7402
impairment,-287542,8407,-10361
finance_costs,-26444,-23201,-19146
income_tax,1729,-533,-1494
depreciation,45541,31903,35319
cash_equivalents,28757,69648,70155


,profit_loss_total,ordinary_shares,earnings_per_share,price_per_share,pe_ratio,ebitda,enterprise_value,ev_ebitda_ratio
FY17,-354331,402771,-87.973,17.585,-0.200,3467,365999,105.566
FY18,-27909,678468,-4.114,24.000,-5.834,19321,358399,18.550
FY19,-37373,858077,-4.355,19.000,-4.363,28947,361135,12.476


### Section 4
> *Create a working/tested R program to load monthly share prices for MMA offshore for the 2018 calendar year. Source the prices online and get them into R in whatever way you this is most useful.*

In [11]:
# Load the timeDate package
library("timeDate")

In [12]:
# Subset stock data to select prices for the 2018 calendar year (in cents)
days_CY18 <- seq(as.Date("2018-01-01"), as.Date("2018-12-31"), by="days")
prices_CY18 <- MRM.AX[days_CY18, "MRM.AX.Adjusted"] * 100

# Use apply.monthly from the timeDate package to select prices at the end of each month
prices_CY18_monthly <- apply.monthly(prices_CY18, head, 1)
names(prices_CY18_monthly) <- "price_monthly"

data.frame(prices_CY18_monthly)

,price_monthly
2018-01-31,28.5
2018-02-28,27.0
2018-03-29,23.0
2018-04-30,24.0
2018-05-31,22.5
2018-06-29,23.0
2018-07-31,24.5
2018-08-31,27.0
2018-09-28,22.0
2018-10-31,25.0


### Section 5
> *Load the FY2018 eps into R and calculate a PE for each month of Calendar year 2018. Share any thoughts you have about why it might change throughout the year.*

In [13]:
# Calculate monthly PE ratio
pe_ratio_monthly <- round(analysis["FY18", "earnings_per_share"] / prices_CY18_monthly, 3)
names(pe_ratio_monthly) <- "pe_ratio_monthly"

# Calculate monthly EV / EBITDA ratios
enterprise_value_monthly <- analysis["FY18", "ordinary_shares"] * prices_CY18_monthly / 100 + 
    financials["noncurrent_liabilities", "FY18"] - financials["cash_equivalents", "FY18"]
ev_ebitda_ratio_monthly <- round(enterprise_value_monthly / analysis["FY18", "ebitda"], 3)
names(ev_ebitda_ratio_monthly) <- "ev_ebitda_ratio"

# Summarise results in a dataframe
analysis_monthly <- data.frame(prices_CY18_monthly, pe_ratio_monthly, ev_ebitda_ratio_monthly)
analysis_monthly

,price_monthly,pe_ratio_monthly,ev_ebitda_ratio
2018-01-31,28.5,-0.144,20.130
2018-02-28,27.0,-0.152,19.603
2018-03-29,23.0,-0.179,18.199
2018-04-30,24.0,-0.171,18.550
2018-05-31,22.5,-0.183,18.023
2018-06-29,23.0,-0.179,18.199
2018-07-31,24.5,-0.168,18.725
2018-08-31,27.0,-0.152,19.603
2018-09-28,22.0,-0.187,17.847
2018-10-31,25.0,-0.165,18.901


### Section 6
> **`a.`** *Doing some research online, what reasons (in order of importance) would you give to explain Forager’s interest in MRM?*

> **`b.`** *Can you highlight any specific risks that MRM poses for Forager as an investment?*